In [12]:
import glob 
import pandas as pd 
import numpy as np 
import os 
from utils import * 
import subprocess
import itertools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
sample_metadata_df = pd.read_csv('sample_metadata.csv')
sample_metadata_df['metat'] = sample_metadata_df.sample_id.str.contains('metat')

ggkbase_id_to_year_map = sample_metadata_df.set_index('ggkbase_id').year.to_dict()
sample_id_to_year_map = sample_metadata_df.set_index('sample_id').year.to_dict()
ggkbase_id_to_sample_id_map = sample_metadata_df.set_index('ggkbase_id').sample_id.to_dict()
sample_id_to_ggkbase_id_map = sample_metadata_df.set_index('sample_id').ggkbase_id.to_dict()

genome_ids = pd.read_csv('./data/genome_metadata.csv').genome_id.values
sample_ids = sample_metadata_df.sample_id.unique()
metat_sample_ids = sample_metadata_df[sample_metadata_df.metat].sample_id.unique()

sample_path_template = '/groups/banfield/sequences/{year}/{ggkbase_id}/raw.d/{ggkbase_id}_trim_clean.{paired_end}.fastq.gz'
ref_genome_dir = '~/data/'
# ref_genome_paths = [os.path.join(ref_genome_dir, f'{genome_id}.fn') for genome_id in genome_ids]

FileNotFoundError: [Errno 2] No such file or directory: 'sample_metadata.csv'

In [3]:
coverm_sample_paths = '' 
for row in sample_metadata_df[~sample_metadata_df.metat].itertuples():
    coverm_sample_paths += sample_path_template.format(ggkbase_id=row.ggkbase_id, paired_end='PE.1', year=row.year) + ' '
    coverm_sample_paths += sample_path_template.format(ggkbase_id=row.ggkbase_id, paired_end='PE.2', year=row.year) + ' '
coverm_sample_paths = coverm_sample_paths.strip()

coverm_fields = 'mean trimmed_mean covered_bases variance count rpkm tpm'

ref_genome_path = os.path.join(ref_genome_dir, f'all.fn')
output_path = f'/home/philippar/data/coverm/all.tsv'
cmd = f'coverm contig -c {coverm_sample_paths} -r {ref_genome_path} --min-read-percent-identity 97 --min-read-aligned-percent 80 --trim-min 5 --trim-max 95 -m {coverm_fields} -t 20 -o {output_path}'
print(f'sbatch --wrap "{cmd}"\n')

sbatch --wrap "coverm contig -c /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_bot_05_17_2025/raw.d/SR-VP_Bioreactor_ck_bot_05_17_2025_trim_clean.PE.1.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_bot_05_17_2025/raw.d/SR-VP_Bioreactor_ck_bot_05_17_2025_trim_clean.PE.2.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_mid_05_17_2025/raw.d/SR-VP_Bioreactor_ck_mid_05_17_2025_trim_clean.PE.1.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_mid_05_17_2025/raw.d/SR-VP_Bioreactor_ck_mid_05_17_2025_trim_clean.PE.2.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_top_05_17_2025/raw.d/SR-VP_Bioreactor_ck_top_05_17_2025_trim_clean.PE.1.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_top_05_17_2025/raw.d/SR-VP_Bioreactor_ck_top_05_17_2025_trim_clean.PE.2.fastq.gz /groups/banfield/sequences/2025/SR-VP_Bioreactor_N_bot_05_17_2025/raw.d/SR-VP_Bioreactor_N_bot_05_17_2025_trim_clean.PE.1.fastq.gz /groups/banfield/sequences/2025/SR-VP_B

In [4]:
# with open('../scripts/bbduk_library_sizes.sh', 'w') as f:
#     for name, sample_id in sample_id_map.items():
#             paired_ends_1_path = sample_path_template.format(paired_end='PE.1', sample_id=sample_id, year=sample_year_map.get(sample_id, 2025))
#             paired_ends_2_path = sample_path_template.format(paired_end='PE.2', sample_id=sample_id, year=sample_year_map.get(sample_id, 2025))
#             output_path = f'/home/philippar/data/bbduk/{name}.txt'
#             cmd = f'bbduk.sh in={paired_ends_1_path} in2={paired_ends_2_path} out=/dev/null stats={output_path} threads=64'
#             f.write(f'sbatch --wrap "{cmd}"\n')

In [5]:
def bbmap_get_command(sample_id:str, genome_id:str, output_dir:str='../data/metat/'):
    ref_genome_path = os.path.join(ref_genome_dir, f'{genome_id}.fn')

    input_path_1 = sample_path_template.format(ggkbase_id=sample_id_to_ggkbase_id_map[sample_id], paired_end='PE.1', year=sample_id_to_year_map[sample_id])
    input_path_2 = sample_path_template.format(ggkbase_id=sample_id_to_ggkbase_id_map[sample_id], paired_end='PE.2', year=sample_id_to_year_map[sample_id])

    # output_dir = os.path.join(output_dir, genome_id)
    output_path = os.path.join(output_dir, f'{sample_id}.bam')

    params = 'pigz=t unpigz=t ambiguous=random minid=0.96 idfilter=0.97 threads=64 out=stdout.sam editfilter=5 out=stdout.sam'
    cmd = f'bbmap.sh {params} in1={input_path_1} in2={input_path_2} ref={ref_genome_path} nodisk | shrinksam | sambam > {output_path}'
    return cmd, output_path


output_dir = '../data/metat'

genome_id = 'all'
# print(f'mkdir -p {os.path.join(output_dir, genome_id)}') # Make sure the output directory exists.
for sample_id in metat_sample_ids:        
    cmd, output_path = bbmap_get_command(sample_id, genome_id, output_dir=output_dir)
    # print(f'sbatch --wrap "{cmd}"')
    print(f'srun {cmd}')



srun bbmap.sh pigz=t unpigz=t ambiguous=random minid=0.96 idfilter=0.97 threads=64 out=stdout.sam editfilter=5 out=stdout.sam in1=/groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_bot_05_17_2025_metaT/raw.d/SR-VP_Bioreactor_ck_bot_05_17_2025_metaT_trim_clean.PE.1.fastq.gz in2=/groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_bot_05_17_2025_metaT/raw.d/SR-VP_Bioreactor_ck_bot_05_17_2025_metaT_trim_clean.PE.2.fastq.gz ref=~/data/all.fn nodisk | shrinksam | sambam > ../data/metat/ck_bottom_2025_metat.bam
srun bbmap.sh pigz=t unpigz=t ambiguous=random minid=0.96 idfilter=0.97 threads=64 out=stdout.sam editfilter=5 out=stdout.sam in1=/groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_mid_05_17_2025_metaT/raw.d/SR-VP_Bioreactor_ck_mid_05_17_2025_metaT_trim_clean.PE.1.fastq.gz in2=/groups/banfield/sequences/2025/SR-VP_Bioreactor_ck_mid_05_17_2025_metaT/raw.d/SR-VP_Bioreactor_ck_mid_05_17_2025_metaT_trim_clean.PE.2.fastq.gz ref=~/data/all.fn nodisk | shrinksam | sambam > ../data/metat/ck_

In [6]:
def featurecounts_get_command(bam_path:str, ref_path:str=None, output_dir:str=None):
    output_file_name = os.path.basename(bam_path).replace('.bam', '')
    output_file_name += '_read_counts'
    output_path = os.path.join(output_dir, output_file_name)
    return f'featureCounts -p -T 64 -g ID -t CDS -a {ref_path} -s 2 -o {output_path} {bam_path}' 

data_dir = '~/data/metat'

for sample_id in metat_sample_ids:
    bam_path = os.path.join(data_dir, f'{sample_id}.bam')
    ref_path = os.path.join(ref_genome_dir, 'all.gff')
    cmd = featurecounts_get_command(bam_path, ref_path, output_dir=data_dir)
    print(cmd)

featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/ck_bottom_2025_metat_read_counts ~/data/metat/ck_bottom_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/ck_middle_2025_metat_read_counts ~/data/metat/ck_middle_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/ck_top_2025_metat_read_counts ~/data/metat/ck_top_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/n_top_2025_metat_read_counts ~/data/metat/n_top_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/n_bottom_2025_metat_read_counts ~/data/metat/n_bottom_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/n_middle_2025_metat_read_counts ~/data/metat/n_middle_2025_metat.bam
featureCounts -p -T 64 -g ID -t CDS -a ~/data/all.gff -s 2 -o ~/data/metat/ck_bottom_2024_metat_read_counts ~/data/metat/ck_bottom_2024_metat.

In [14]:
genome_ids

array(['mp_1', 'mp_2', 'mp_3', 'mp_4', 'mp_5', 'mp_6', 'mp_7', 'mp_8',
       'mp_9', 'mp_10', 'mp_11', 'mp_12', 'mp_13', 'mp_14', 'mp_15',
       'mp_16', 'mp_17', 'mp_18', 'mp_19', 'mp_20', 'jupiter_mini_borg_1',
       'jupiter_mini_borg_2', 'jupiter_mini_borg_3',
       'jupiter_mini_borg_4', 'jupiter_mini_borg_6',
       'jupiter_mini_borg_7', 'jupiter_mini_borg_8',
       'jupiter_mini_borg_9', 'saturn_mini_borg_1', 'saturn_mini_borg_2',
       'saturn_mini_borg_3', 'saturn_mini_borg_4',
       'unclassified_mini_borg', 'unclassified_borg', 'amethyst_borg',
       'oxblood_borg', 'pink_borg', 'purple_borg', 'rose_borg',
       'vermilion_borg', 'mercury_mini_borg', 'saturn_mini_borg_like',
       'ruby_borg_related', 'black_borg', 'linear_ece_19kb'], dtype=object)

In [15]:

def interproscan_get_command(input_path:str, output_dir:str='~/data/interproscan/'):
    id_ = os.path.basename(input_path).split('.')[0]
    output_path = os.path.join(output_dir, f'{id_}.tsv')
    cmd = f'interproscan.sh -i {input_path} -o {output_path} -f tsv'
    return cmd

with open('../scripts/interproscan_annotation.sh', 'w') as f:
    for genome_id in genome_ids:
        input_path = f'~/data/{genome_id}.fa'
        cmd = interproscan_get_command(input_path)
        f.write(f'sbatch --wrap "{cmd}"' + '\n')

In [8]:
def kofamscan_get_command(input_path:str, output_dir:str='~/data/kofamscan/'):
    id_ = os.path.basename(input_path).split('.')[0]
    output_path = os.path.join(output_dir, f'{id_}.tsv')
    profiles_path = '/shared/db/kegg/kofam/latest/profiles'
    ko_list_path = '/shared/db/kegg/kofam/latest/metadata/ko_list'
    cmd = f'/shared/software/kofamscan/latest/exec_annotation {input_path} -o {output_path} -f detail-tsv -E 0.1 --cpu 16 -k {ko_list_path} -p {profiles_path}'
    cmd = f'sbatch --wrap "{cmd}"'
    return cmd

with open('../scripts/kofamscan_annotation.sh', 'w') as f:
    for genome_id in genome_ids:
        input_path = f'~/data/{genome_id}.fa'
        cmd = kofamscan_get_command(input_path)
        f.write(cmd + '\n')

In [9]:
target_names = [f'mp_{i + 1}' for i in range(5)]
metat_sample_ids = sample_metadata_df[sample_metadata_df.sample_id.str.contains('metat')].sample_id.values

for target_name, sample_id in itertools.product(target_names, metat_sample_ids):
    awk_pattern = '\'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}\''
    input_path = f'~/data/metat/{target_name}/{sample_id}.bam'
    output_path = f'~/data/metat/{target_name}/{sample_id}.csv'
    cmd = f'samtools view -F 4  {input_path} | awk ' + awk_pattern + f' > {output_path}'
    print(cmd)

samtools view -F 4  ~/data/metat/mp_1/ck_bottom_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}' > ~/data/metat/mp_1/ck_bottom_2025_metat.csv
samtools view -F 4  ~/data/metat/mp_1/ck_middle_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}' > ~/data/metat/mp_1/ck_middle_2025_metat.csv
samtools view -F 4  ~/data/metat/mp_1/ck_top_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}' > ~/data/metat/mp_1/ck_top_2025_metat.csv
samtools view -F 4  ~/data/metat/mp_1/n_top_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}' > ~/data/metat/mp_1/n_top_2025_metat.csv
samtools view -F 4  ~/data/metat/mp_1/n_bottom_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start";OFS=","} {print $1, $4, $5}' > ~/data/metat/mp_1/n_bottom_2025_metat.csv
samtools view -F 4  ~/data/metat/mp_1/n_middle_2025_metat.bam | awk 'BEGIN{print "read_id,contig_id,start"

In [10]:
metat_sample_ids

array(['ck_bottom_2025_metat', 'ck_middle_2025_metat',
       'ck_top_2025_metat', 'n_top_2025_metat', 'n_bottom_2025_metat',
       'n_middle_2025_metat', 'ck_bottom_2024_metat', 'n_top_2024_metat',
       'n_bottom_2024_metat', 'n_middle_2024_metat'], dtype=object)